In [7]:
import re
import datetime
import pandas

In [8]:
from IPython.display import display, HTML

In [9]:
results = pandas.read_csv('~/Downloads/draftkings-contest-entry-history.csv')

In [21]:
results['year'] = results['Contest_Date_EST'].apply(lambda k: k[0:4])
results['month'] = results['Contest_Date_EST'].apply(lambda k: k[0:7])
results['entry_fee'] = results['Entry_Fee'].apply(lambda k: float(re.sub('\$|,', '', k)))
results['recoup_nonticket'] = results['Winnings_Non_Ticket'].apply(lambda k: float(k.replace('$','').replace(',','')))
results['recoup_ticket']    = results['Winnings_Ticket'].apply(lambda k: float(k.replace('$','').replace(',','')))
results['recoup']           = results['recoup_nonticket'] + results['recoup_ticket']
results['pnl']              = results['recoup'] - results['entry_fee']

In [1]:
sport_years = results.groupby(['Sport','year','month'], as_index=False).aggregate({'entry_fee': 'sum','recoup': 'sum','pnl':'sum'})
sport_years.sort_values(['pnl',], ascending=False)

In [2]:
sport_years.groupby(['year'], as_index=False).aggregate({'pnl':'sum'})

In [29]:
nba = sport_years[sport_years['Sport']=='NBA']

In [3]:
nba